Let's start by setting up some simulation parameters for the market. We'll do calculations in WEI to keep close to the original code. We will convert to ETH as necessary. As a preliminary step, let's define some market constants

In [21]:
WEI = 1
GWEI = 10**9
WHOLE = 10**18

def wei_to_whole(amount):
    return amount // 10**18

def gwei_to_whole(amount):
    return amount // 10**9

def gwei_to_wei(amount):
    return amount * 10**9

def wei_to_gwei(amount):
    return amount // 10**9

def convert_investment_price(price):
    """Investment price is ETH wei / MKT gwei. Convert to ETH whole / MKT whole"""
    # ETH gwei / MKT gwei
    price_gwei_to_gwei = wei_to_gwei(price)
    # Note units cancel
    price_whole_to_whole = price_gwei_to_gwei
    return price_whole_to_whole

Here are the marke.t parmeters. The rest of the notebook is designed so that if you change these parameters, the rest of the notebook should execute correctly

In [22]:
PRICE_FLOOR = .001 * GWEI
SPREAD = 110 # 10 %
LIST_REWARD = .000025 * WHOLE # MKT
STAKE = .01 * WHOLE # MKT
VOTE_BY = 7 # Days. Are units right?
BACKEND_PAYMENT = .25
MAKER_PAYMENT = .25
RESERVE_PAYMENT = 1 - BACKEND_PAYMENT - MAKER_PAYMENT
COST_PER_BYTE = 100 * GWEI # ETH

Let's have the market creator create the initial data market.

In [23]:
RESERVE = 0 # Initial reserve is empty
MARKET_TOTAL = 0 # No Market tokens at start

# Creator does their thing
CREATOR_BLOCK_SIZE = 1 * WHOLE # MKT
MARKET_TOTAL += CREATOR_BLOCK_SIZE
# Creator deposits 100 ETH in reserve
RESERVE += 100 * WHOLE # ETH
print("MARKET_TOTAL: %d, RESERVE: %d" % (wei_to_whole(MARKET_TOTAL), wei_to_whole(RESERVE)))

MARKET_TOTAL: 1, RESERVE: 100


Now some investors join the market.

In [28]:
INVESTMENT = 100 * WHOLE # ETH
print("INVESTMENT: %d" % wei_to_whole(INVESTMENT))
INVESTMENT_PRICE = PRICE_FLOOR + ((SPREAD * RESERVE * GWEI) / (100 * MARKET_TOTAL))
print("INVESTMENT_PRICE (ETH wei for 1 MKT gwei): %d" % INVESTMENT_PRICE)
print("INVESTMENT_PRICE (ETH whole for 1 MKT whole): %d" % convert_investment_price(INVESTMENT_PRICE))
MINTED = (INVESTMENT // INVESTMENT_PRICE) * GWEI # Units of GWEI
print(INVESTMENT // INVESTMENT_PRICE)
print(MINTED)
print("MINTED: %d" % wei_to_whole(MINTED))
MARKET_TOTAL += gwei_to_wei(MINTED)
RESERVE += INVESTMENT
print("MINTED: %d, MARKET_TOTAL: %d, RESERVE: %d" % (wei_to_whole(MINTED), wei_to_whole(MARKET_TOTAL), wei_to_whole(RESERVE)))

100000000000000000000
INVESTMENT: 100
INVESTMENT_PRICE (ETH wei for 1 MKT gwei): 1000000
INVESTMENT_PRICE (ETH whole for 1 MKT whole): 0
99999999816652.0
9.9999999816652e+22
MINTED: 99999
MINTED: 99999, MARKET_TOTAL: 399976713984291, RESERVE: 600
